### Hopping and Tumbling Windows

In this demonstration we'll see how to create Tables with windowing enabled. Let's start by creating a `clickevent` stream based on the
kafka topic `com.udacity.streams.clickevents`:

```
CREATE STREAM clickevents
  (email VARCHAR,
   timestamp VARCHAR,
   uri VARCHAR,
   number INTEGER)
  WITH (KAFKA_TOPIC='com.udacity.streams.clickevents',
        VALUE_FORMAT='JSON');
```

### Tumbling Windows

Let's create a tumbling clickevents table, where the window size is 30 seconds.

```
CREATE STREAM clickevents_tumbling AS
  SELECT * FROM clickevents
  WINDOW TUMBLING (SIZE 30 SECONDS);
```

Question to myself: why is it usefull to create a new stream in this way? shouldn't there be a group by specified? Something like:

```
CREATE TABLE clickevents_tumbling5 AS
  SELECT uri, count(number) AS sum_number FROM clickevents
  WINDOW TUMBLING (SIZE 30 SECONDS)
  WHERE uri LIKE 'http://www.bu%'
  GROUP BY uri;
```

Note that now we need to create a table instead of a stream! When we look at the content of the table we see

` ksql> select * from clickevents_tumbling5; `

```
1639492806955 | http://www.butler.com/tag/wp-content/wp-content/register/ : Window{start=1639492800000 end=-} | http://www.butler.com/tag/wp-content/wp-content/register/ | 1
1639492831392 | http://www.butler.com/tag/wp-content/wp-content/register/ : Window{start=1639492830000 end=-} | http://www.butler.com/tag/wp-content/wp-content/register/ | 1
1639492850571 | http://www.burgess.com/login/ : Window{start=1639492830000 end=-} | http://www.burgess.com/login/ | 1
1639492854226 | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp : Window{start=1639492830000 end=-} | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp | 1
1639492858350 | http://www.butler.com/tag/wp-content/wp-content/register/ : Window{start=1639492830000 end=-} | http://www.butler.com/tag/wp-content/wp-content/register/ | 2
1639492884661 | http://www.bush-watts.biz/ : Window{start=1639492860000 end=-} | http://www.bush-watts.biz/ | 1
1639492882637 | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp : Window{start=1639492860000 end=-} | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp | 1
1639492893358 | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp : Window{start=1639492890000 end=-} | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp | 1
1639492934916 | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp : Window{start=1639492920000 end=-} | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp | 1
1639492956455 | http://www.butler.com/tag/wp-content/wp-content/register/ : Window{start=1639492950000 end=-} | http://www.butler.com/tag/wp-content/wp-content/register/ | 1
1639492971446 | http://www.burgess.com/login/ : Window{start=1639492950000 end=-} | http://www.burgess.com/login/ | 1
1639492973422 | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp : Window{start=1639492950000 end=-} | http://www.bush-rivera.com/wp-content/categories/wp-content/login.jsp | 1
```

To understand the columns use `describe`

`describe clickevents_tumbling5;`

```
Name                 : CLICKEVENTS_TUMBLING5
 Field      | Type                      
----------------------------------------
 ROWTIME    | BIGINT           (system) 
 ROWKEY     | VARCHAR(STRING)  (system) 
 URI        | VARCHAR(STRING)           
 SUM_NUMBER | BIGINT                    
----------------------------------------
For runtime statistics and query details run: DESCRIBE EXTENDED <Stream,Table>;
ksql> 
```

### Hopping Windows

Now we can create a Table with a hopping window of 30 seconds with 5 second increments.

```
CREATE TABLE clickevents_hopping AS
  SELECT uri FROM clickevents
  WINDOW HOPPING (SIZE 30 SECONDS, ADVANCE BY 5 SECONDS)
  WHERE uri LIKE 'http://www.b%'
  GROUP BY uri;
```

The above window is 30 seconds long and advances by 5 second. If you query the table you will see
the associated window times!

### Session Windows

Finally, lets see how session windows work. We're going to define the session as 5 minutes in
order to group many events to the same window

```
CREATE TABLE clickevents_session AS
  SELECT uri FROM clickevents
  WINDOW SESSION (5 MINUTES)
  WHERE uri LIKE 'http://www.b%'
  GROUP BY uri;
```
